In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import h5py
subhalo = h5py.File('/store/clues/HESTIA/RE_SIMS/8192/GAL_FOR/17_11/output_2x2.5Mpc/groups_127/fof_subhalo_tab_127.0.hdf5','r')

def printname(name):
    print (name)
    
subhalo.visit(printname)

Config
Group
Group/GroupBHMass
Group/GroupBHMdot
Group/GroupCM
Group/GroupFirstSub
Group/GroupGasMetalFractions
Group/GroupGasMetallicity
Group/GroupLen
Group/GroupLenType
Group/GroupMass
Group/GroupMassType
Group/GroupNsubs
Group/GroupPos
Group/GroupRadioLuminosity
Group/GroupSFR
Group/GroupStarMetalFractions
Group/GroupStarMetallicity
Group/GroupVel
Group/GroupWindMass
Group/GroupXrayLuminosity
Group/Group_M_Crit200
Group/Group_M_Crit500
Group/Group_M_Mean200
Group/Group_M_TopHat200
Group/Group_R_Crit200
Group/Group_R_Crit500
Group/Group_R_Mean200
Group/Group_R_TopHat200
Header
IDs
Parameters
Subhalo
Subhalo/SubhaloBHMass
Subhalo/SubhaloBHMdot
Subhalo/SubhaloBfldDisk
Subhalo/SubhaloBfldHalo
Subhalo/SubhaloCM
Subhalo/SubhaloGasMetalFractions
Subhalo/SubhaloGasMetalFractionsHalfRad
Subhalo/SubhaloGasMetalFractionsMaxRad
Subhalo/SubhaloGasMetalFractionsSfr
Subhalo/SubhaloGasMetalFractionsSfrWeighted
Subhalo/SubhaloGasMetallicity
Subhalo/SubhaloGasMetallicityHalfRad
Subhalo/SubhaloGasMet

In [60]:
%%time
import gc
import joblib
import h5py
import pathlib
import numpy as np

dir_path = pathlib.Path('/store/clues/HESTIA/RE_SIMS/8192/GAL_FOR/17_11/output_2x2.5Mpc/snapdir_127')


def load(path):
    with h5py.File(path,'r') as snap:
        pstr = snap['PartType4/Coordinates'][()]
        pdrk = snap['PartType1/Coordinates'][()]
        pgas = snap['PartType0/Coordinates'][()]
        mstr = snap['PartType4/Masses'][()]
        mdrk = snap['PartType1/Masses'][()]
        mgas = snap['PartType0/Masses'][()]
        
        a = snap.get('Header').attrs['Time']
        h = snap.get('Header').attrs['HubbleParam']


    return {'StrPos' : pstr, 
            'DrkPos' : pdrk, 
            'GasPos' : pgas, 
            'StrMass': mstr,
            'DrkMass': mdrk,
            'GasMass': mgas,
            'aexp'   : a,
            'h'      : h}

with joblib.Parallel(n_jobs=8) as P:
    results = P(
        joblib.delayed(load)(path)
        for path in dir_path.glob('*.hdf5'))
    
# results = np.vstack(results)

gc.collect()

CPU times: user 12.5 s, sys: 42.4 s, total: 54.9 s
Wall time: 1min 6s


142

In [61]:
aexp = results[0]['aexp']
h    = results[0]['h']

pos_str  = np.concatenate([r['StrPos']for r in results])
pos_drk  = np.concatenate([r['DrkPos']for r in results])
pos_gas  = np.concatenate([r['GasPos']for r in results])

pos_str = pos_str*aexp*1000/h
pos_drk = pos_drk*aexp*1000/h
pos_gas = pos_gas*aexp*1000/h

mass_str = np.concatenate([r['StrMass']for r in results])
mass_drk = np.concatenate([r['DrkMass']for r in results])
mass_gas = np.concatenate([r['GasMass']for r in results])

mstr = mass_str*1e10/h
mdrk = mass_drk*1e10/h
mgas = mass_gas*1e10/h


In [62]:
%%time
import gc
import joblib
import h5py
import pathlib
import numpy as np

dir_path = pathlib.Path('/store/clues/HESTIA/RE_SIMS/8192/GAL_FOR/17_11/output_2x2.5Mpc/groups_127')


def load(path):
    with h5py.File(path,'r') as snap:
        sub_mass = snap['Subhalo/SubhaloMass'][()]
        sub_pos  = snap['Subhalo/SubhaloPos'][()]
        
        Omega0   = snap.get('Header').attrs['Omega0']
        OmegaL   = snap.get('Header').attrs['OmegaLambda']
        Redshift = snap.get('Header').attrs['Redshift']
    
    return {'SubMass': sub_mass, 
            'SubPos' : sub_pos,
            
            'Omega0'  : Omega0,
            'OmegaL'  : OmegaL,
            'Redshift': Redshift}

with joblib.Parallel(n_jobs=8) as P:
    results = P(
        joblib.delayed(load)(path)
        for path in dir_path.glob('*.hdf5'))
    
# results = np.vstack(results)
gc.collect()

CPU times: user 66 ms, sys: 1.51 s, total: 1.57 s
Wall time: 1.57 s


142

In [63]:
Smass = np.concatenate([r['SubMass']for r in results])
Spos  = np.concatenate([r['SubPos']for r in results])

sort = np.argsort(Smass)

SMASS = Smass[sort][::-1]*1e10/h
SPOS  = Spos*aexp*1000/h

In [65]:
%%time

import numpy as np
import gc

G = 4.299e-6 #kpc*(km/s)*Msol^-1

H0 = 100*h #km/s/Mpc

omega_matter = results[0]['Omega0']
omega_lambda = results[0]['OmegaL']
omega0 = omega_matter + omega_lambda

z = results[0]['Redshift']

nsub = 500
Mvir = np.zeros(nsub)
Rvir = np.zeros(nsub)

for i in range(nsub):
    
    xstr = pos_str[:,0] - SPOS[i,0]
    ystr = pos_str[:,1] - SPOS[i,1]
    zstr = pos_str[:,2] - SPOS[i,2]
    rstr = np.sqrt(xstr**2 + ystr**2 + zstr**2)

    xdrk = pos_drk[:,0] - SPOS[i,0]
    ydrk = pos_drk[:,1] - SPOS[i,1]
    zdrk = pos_drk[:,2] - SPOS[i,2]
    rdrk = np.sqrt(xdrk**2 + ydrk**2 + zdrk**2)
    
    xgas = pos_gas[:,0] - SPOS[i,0]
    ygas = pos_gas[:,1] - SPOS[i,1]
    zgas = pos_gas[:,2] - SPOS[i,2]
    rgas = np.sqrt(xgas**2 + ygas**2 + zgas**2)
    
    st, = np.where(rstr < 1000)
    dm, = np.where(rdrk < 1000)
    gs, = np.where(rstr < 1000)
    
    r = np.concatenate([rstr[st], rgas[gs], rdrk[dm]])
    m = np.concatenate([mstr[st], mgas[gs], mdrk[dm]])

    psort = np.argsort(r)

    vol  = 4./3 * np.pi * r[psort]**3
    mass = np.cumsum(m[psort])
    rho  = mass / vol

    H0_kpc = H0/1000 #paso a kpc
    
    Ht = H0_kpc*np.sqrt(omega_lambda+(1-omega0)*(1+z)**2+omega_matter*(1+z)**3)
    
    rho_crit = 3*Ht**2 / (8*np.pi*G)

    mask, = np.where(rho < 200*rho_crit)
    
    Mvir[i] = mass[mask][0]
    Rvir[i] = r[psort][mask][0] 
    
    print (i)
#     print ('subhalo', str('%03d'%i), 'Mvir = ', str('%.4E'%Mvir[i]), 'Rvir = ', str('%4.8f'%Rvir[i]))
    
    gc.collect()
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [69]:
msort = np.argsort(Mvir)
shID = msort

data = np.ndarray([nsub, 3])
data[:,0] = Mvir[msort][::-1]
data[:,1] = Rvir[msort][::-1]
data[:,2] = shID[::-1]

np.savetxt('../_data/Mvir_Rvir.dat', data, fmt = ('%.5E','%12.8f', '%03d'))

In [59]:
for i in range(11):
    print ('%.3E'%Mvir[i])

1.959E+12
1.833E+11
1.568E+12
1.188E+11
2.816E+10
1.538E+11
1.912E+12
1.796E+12
3.358E+09
1.845E+10
3.872E+09


In [50]:
msort = np.argsort(Mvir)
print (msort[::-1])

[ 0  6  7  2  1  5  3  4  9 10  8 39 31 32 33 34 35 36 37 38 44 40 41 42
 43 29 45 46 47 30 24 28 18 11 12 13 14 15 16 17 19 27 20 21 22 23 98 25
 26 48 99 50 75 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 76 74 51 73 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69
 70 71 72 49]


In [17]:
(1.88e-29)*h**2 *(3.086e21)**3 / (1.989e33) 

127.58094145367797

In [39]:
print (rho_crit*200)

25504.62350081994


In [41]:
print (r[psort][mask])

[  259.55519224   259.58658917   259.59027663 ... 90929.78325109
 91228.94830954 91241.25829807]


In [ ]:
for i in range(100):
    print (SMASS[i], Mvir[i])

In [ ]:

r = np.concatenate(rstr,rgas,rdrk)
m = np.concatenate(mstr,mgas,mdrk)

psort = np.argsort(r)

vol  = 4./3 * np.pi * r[psort]**3
mass = m[psort]
rho  = mass / vol

rho_crit = 3*H**2 / (8*np.pi*G)

mask, = np.where(rho < 200*rho_crit)
Mvir = mvir[-1]

In [ ]:
h = 0.6777
G = 4.299e-6 #kpc*(km/s)*Msol^-1
a0 = 1
H0 = h*100
omega_lambda = 0.682
omega_matter = 0.318
omega0 = omega_matter + omega_lambda

z = a0/aexp[isnap] - 1.
Ht = H0*np.sqrt(omega_lambda+(1-omega0)*(1+z)**2+omega_matter*(1+z)**3)
time = tiempo.conv(z, h, omega_lambda, omega_matter)

#---aca paso las coordenadas respecto al centro de la galaxia------
xstr = (pstr[:,0]-cm[0])*aexp[isnap]/h
ystr = (pstr[:,1]-cm[1])*aexp[isnap]/h
zstr = (pstr[:,2]-cm[2])*aexp[isnap]/h

v_x = vel[:,0] *np.sqrt(aexp[isnap]) + Ht *xstr/1000.
v_y = vel[:,1] *np.sqrt(aexp[isnap]) + Ht *ystr/1000.
v_z = vel[:,2] *np.sqrt(aexp[isnap]) + Ht *zstr/1000.